# housingkaggle

Use the "Run" button to execute the code.

In [15]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline

In [17]:
import pandas as pd
import numpy as np
# Importing the Boston Housing dataset
from sklearn.datasets import load_boston
boston = load_boston()
# Initializing the dataframe
data = pd.DataFrame(boston.data)
# See head of the dataset
data.head()

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [18]:
#Adding the feature names to the dataframe
data.columns = boston.feature_names
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [19]:
#Adding target variable to dataframe
data['PRICE'] = boston.target

In [20]:
len(data)

506

In [22]:
val_size = 200
train_size = len(data) - val_size

train_ds, val_ds = random_split(data, [train_size, val_size])
len(train_ds), len(val_ds)

(306, 200)

In [23]:
batch_size=12

In [24]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)

/opt/conda/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [32]:
df = pd.DataFrame(boston.data)
df.columns = boston.feature_names
df['Price'] = boston.target
df.head()
data = df[df.columns[:-1]]
data = data.apply(
    lambda x: (x - x.mean()) / x.std()
)

data['Price'] = df.Price
import numpy as np

X = data.drop('Price', axis=1).to_numpy()
Y = data['Price'].to_numpy()

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(354, 13)
(152, 13)
(354,)
(152,)


In [38]:
import torch


In [41]:
n_train = X_train.shape[0]
X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
Y_train = torch.tensor(Y_train, dtype=torch.float).view(-1, 1)
Y_test = torch.tensor(Y_test, dtype=torch.float).view(-1, 1)

In [42]:
w_num = X_train.shape[1]
net = torch.nn.Sequential(
    torch.nn.Linear(w_num, 1)
)

torch.nn.init.normal_(net[0].weight, mean=0, std=0.1)
torch.nn.init.constant_(net[0].bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

In [43]:
datasets = torch.utils.data.TensorDataset(X_train, Y_train)


In [44]:
train_iter = torch.utils.data.DataLoader(datasets, batch_size=10, shuffle=True)

In [45]:
loss = torch.nn.MSELoss()

In [46]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.05)

In [48]:
num_epochs = 100
for epoch in range(num_epochs):
    for x, y in train_iter:
        output = net(x)
        l = loss(output, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print("epoch {} loss: {:.4f}".format(epoch + 1, l.item()))

epoch 1 loss: 67.9656
epoch 2 loss: 8.3053
epoch 3 loss: 71.1579
epoch 4 loss: 21.3399
epoch 5 loss: 38.8212
epoch 6 loss: 3.9549
epoch 7 loss: 2.2750
epoch 8 loss: 24.6920
epoch 9 loss: 21.6662
epoch 10 loss: 0.4337
epoch 11 loss: 26.2334
epoch 12 loss: 30.0237
epoch 13 loss: 9.9555
epoch 14 loss: 41.5899
epoch 15 loss: 75.3308
epoch 16 loss: 37.1647
epoch 17 loss: 4.4569
epoch 18 loss: 210.0172
epoch 19 loss: 5.8688
epoch 20 loss: 19.2362
epoch 21 loss: 20.7629
epoch 22 loss: 6.3096
epoch 23 loss: 22.2836
epoch 24 loss: 16.9269
epoch 25 loss: 3.9564
epoch 26 loss: 65.9863
epoch 27 loss: 37.3777
epoch 28 loss: 7.9144
epoch 29 loss: 5.9223
epoch 30 loss: 4.7832
epoch 31 loss: 11.9194
epoch 32 loss: 12.3204
epoch 33 loss: 22.4626
epoch 34 loss: 17.0198
epoch 35 loss: 34.1006
epoch 36 loss: 38.7012
epoch 37 loss: 7.4190
epoch 38 loss: 23.1974
epoch 39 loss: 22.4841
epoch 40 loss: 12.4240
epoch 41 loss: 8.0396
epoch 42 loss: 12.2588
epoch 43 loss: 10.2737
epoch 44 loss: 30.1885
epoch 45 l

In [49]:
print(loss(net(X_test), Y_test).item())

23.6146183013916
